### Dota Hero Grid Editor

Hero grids are stored in the following `.json` file in the steam folder:\
`C:\Program Files (x86)\Steam\userdata\YOUR_FRIEND_ID\570\remote\cfg\hero_grid_config.json`

Let's copy it into 
* our working directory - as **a dump file** to play with
* subfolder `./saved` of it^ to keep as **a backup file** in case we do something illegal

In [1]:
# pretty self-explanatory but dota_utils give functions 
# to get hero names by their ids and vise versa
from dota_utils import id_by_name, name_by_id

import json
import shutil
from config import DOTA_FRIENDID

steam_cfg_loc = f'C:\\Program Files (x86)\\Steam\\userdata\\{DOTA_FRIENDID}\\570\\remote\\cfg'
json_rel_loc = '\\hero_grid_config.json'  # rel- relative  
src_steam_cfg = f'{steam_cfg_loc}{json_rel_loc}'  # source
dst_dump = '.'  # destination for dump
dst_backup = './saved'  # destination for backup
shutil.copy2(src_steam_cfg, dst_dump)
_ = shutil.copy2(src_steam_cfg, dst_backup) # _ = just so it doesnt print into output

### READ WRITE CELLS

In [2]:
# READ FROM DUMP FILE
with open('hero_grid_config.json') as json_file:
    data = json.load(json_file)

def write_and_copy():
    # WRITE INTO DUMP FILE 
    with open('hero_grid_config.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    # COPY DUMP FILE BACK TO STEAM_CFG_LOC FOLDER WHEN WE ARE DONE
    src_dump = dst_dump + json_rel_loc
    shutil.copy2(src_dump, steam_cfg_loc)

### Info to plan the Hero Grid
`data` - is the dictionary that is gonna be dumped into the json file back. 

```python
MAX_X = 1200
MAX_Y = 598
```
remember that this is not "pixels" - it just means that hero grid is overall 1200x598 in its own Dota 2 coordinates for the json file.

`data['configs'][0]` is my main Draft grid\
`data['configs'][1]` is Dota Plus levels grid

In [3]:
from typing import Dict, NamedTuple 

MAX_X, MAX_Y = 1200, 598

class P(NamedTuple):
    x: int | float
    y: int | float
    w: int | float
    h: int | float

class Grid:
    def __init__(self, config_index: int, categories: Dict[str, P]) -> None:
        self.config_index: int = config_index
        self.categories: Dict[str, P] = categories

    def update_categories(self):
        for category in data['configs'][self.config_index]['categories']:
            try:
                pos = self.categories[category['category_name']]
            except KeyError as error:
                raise KeyError(
                    f'Category with name "{error}" does not exist in this hero grid.'
                    'Please add this category into your actual grid in Dota 2 client or in file yourself.'
                )
            category['x_position'] = pos.x
            category['y_position'] = pos.y
            category['width'] = pos.w
            category['height'] = pos.h

### Dota Plus Levels Grid

In [4]:
# changeable constants
left_limit_x = 400  # the line between my left and right grid parts
height = 100 
delta = 20
grind_h = 60  # the very last small row
bronze5_sep_x = 200
bronze_delta = 20
bronze5_h = 160

# Categories 
dota_plus_grid_categories = {  # match these names with the ones you have in the hero grid 
    'Grandmaster': P(x=0,  y=0, w=left_limit_x/2, h=height),
    'Master': P(x=left_limit_x/2, y=0, w=left_limit_x/2, h=height),
    'Platinum': P(x=0, y=height+delta, w=left_limit_x, h=height),
    'Gold': P(x=0, y=2*(height+delta), w=left_limit_x, h=height),
    'Silver': P(x=0, y=3*(height+delta), w=left_limit_x, h=MAX_Y - 3*(height+delta)),
    'Bronze 5, >600xp': P(x=left_limit_x, y=0, w=bronze5_sep_x, h=bronze5_h),
    'Bronze 5, <600xp': P(x=left_limit_x + bronze5_sep_x, y=0, w=MAX_X-left_limit_x-bronze5_sep_x, h=bronze5_h),
    'Bronze 4-': P(x=left_limit_x, y=bronze_delta+bronze5_h, w=MAX_X-left_limit_x, h=MAX_Y-bronze_delta-delta-grind_h-bronze5_h),
    'Grind': P(x=left_limit_x, y=MAX_Y-grind_h, w=MAX_X-left_limit_x, h=grind_h)
}

class DotaPlusGrid(Grid):
    def __init__(self) -> None:
        super().__init__(config_index=1, categories=dota_plus_grid_categories)

    def sort_by_dota_plus_xp(self):
        # todo: hmm, idk how to do it considering 
        # I have data private
        # there is a way from Stratz to get it though
        pass

dota_plus_grid = DotaPlusGrid()
dota_plus_grid.update_categories()
write_and_copy()

In [5]:
data['configs'][0]['categories'][1]

{'category_name': 'pos2',
 'x_position': 0,
 'y_position': 105.5,
 'width': 577,
 'height': 95,
 'hero_ids': [21, 39, 46, 43, 25, 22, 74]}

### My default picking screen Grid

In [11]:
amount_heroes_left_column = ahl = 10
left_limit_x = ahl * 58 - 3

# positions
height = 95
rows = 5  # 5 positions in dota 
ban_height = 69
ban_space = 12
delta_pos = (MAX_Y - height * rows - ban_height - ban_space) / (rows - 1)
print(delta_pos)

# attributes
delta_attr = 28
max_attr_height = MAX_Y - 1  # -1 bcs idk bugs out :D 
amount_lines = {'Str': 3, 'Agi': 2, 'Int': 2}
total_lines = sum(amount_lines.values())
attr_heights = [(max_attr_height-2*delta_attr)/total_lines * i for i in amount_lines.values()]
print(attr_heights) 
# idk how to get desired result with math
attr_heights = [221, 160, 160]

default_roles_grid_categories = {
    f'pos{i+1}': P(x=0, y=0+(height+delta_pos)*i, w=left_limit_x, h=height)
    for i in range(rows)
} | {
    name: P(x=left_limit_x, y=sum(attr_heights[0:i])+i*delta_attr, w=MAX_X-left_limit_x, h=attr_heights[i])
    for i, name in enumerate(amount_lines.keys())
} | {
    name: P(
        x=i*17/72*left_limit_x, 
        y=MAX_Y - ban_height, 
        w=2/9*left_limit_x, h=ban_height
    )
    for i, name in enumerate(['bans', 'Turbo bans'])
} | {
    'Grind/Arcana/D+/Style/Cavern': P(
        x=34/72*left_limit_x, 
        y=MAX_Y - ban_height, 
        w=38/72*left_limit_x, h=ban_height
    )
}

class DefaultRolesGrid(Grid):
    def __init__(self) -> None:
        super().__init__(config_index=0, categories=default_roles_grid_categories)

    def sort_attr_categories_by_name(self):
        for category in data['configs'][self.config_index]['categories']:
            if category['category_name'] in ['Str', 'Agi', 'Int']:
                hero_names = [name_by_id(i) for i in category['hero_ids']]
                new_names = sorted(hero_names, key=str.casefold)

                new_ids = [id_by_name(n) for n in new_names]
                category['hero_ids'] = new_ids


default_grid = DefaultRolesGrid()
default_grid.update_categories()
default_grid.sort_attr_categories_by_name()
write_and_copy()

10.5
[231.8571428571429, 154.57142857142858, 154.57142857142858]
